In [1]:
# ----------------------------------------------------------------------------------------- #
# STEP00_v1
    # A. Find all the brw files.
    # B. Get the experimenal variables from the brw file
    # C. Define the appropiate segment size for cutting the dataset
    # D. Cut each segment form the unmodified dataset, transform to raw voltage data ( μV )
    # E. Detect initial channels to be discarded by saturation level
    # F. Make initial figures
    # G. Save segments and evaluation values
# ----------------------------------------------------------------------------------------- #
# * This value can or should be modified
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
# Custom made functions
# ----------------------------------------------------------------------------------------- #
# Path to the folder where the custom functions are storage
# absolute path of the folder where the modules are stored
PATHMODULE = joinpath( homedir( ), "path-example-example", "Code" ); #                  *
push!( LOAD_PATH, PATHMODULE );
using STEP00_v1
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
# Julia's native packages
# ----------------------------------------------------------------------------------------- #
using JLD2
using StatsBase
using Plots
using Measures
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
# A. Searching for directories and files to use
# ----------------------------------------------------------------------------------------- #
PATHSTART = joinpath( homedir( ), "path-example" ); #                                     *
_, FILESBRW = FindDirsFiles( PATHSTART, ".brw" );
for i in 1:length( FILESBRW )
    f = basename( FILESBRW[ i ] );
    println( string( "$i - $f" ) );
end
BRWs = length( FILESBRW );
# ----------------------------------------------------------------------------------------- #
# Select a file to work with from now on.
# ----------------------------------------------------------------------------------------- #
brw = 0;
while brw > BRWs || brw < 1
    print( "Enter the file number: [ between 1 and $BRWs ]" );
    brw = parse( Int, readline( ) );
end
FILEBRW = FILESBRW[ brw ];
println( "\nFile selected: $FILEBRW" ); 
# ----------------------------------------------------------------------------------------- #

[ Info: Precompiling STEP00_v1 [top-level]


1 - 060915s01.brw
2 - 0Ca.brw
3 - 4AP.brw
4 - 700_3-3.brw
5 - 700_3-4.brw
6 - 700_4-3.brw
7 - Phase_01.brw
8 - Phase_02.brw
9 - Phase_02_2.brw
10 - Phase_06.brw
11 - Phase_08.brw
12 - Spontaneous.brw
13 - estriado_slice1_1004AP.brw
14 - estriado_slice1_1504AP.brw
15 - estriado_slice1_2004AP.brw
16 - estriado_slice2_NMDA.brw
17 - estriado_slice2_NMDA_1004AP.brw
18 - estriado_slice2_control.brw
19 - lesion.brw
Enter the file number: [ between 1 and 19 ]stdin> 19

File selected: /home/LBitn/path-example/Data/BRWs/lesion.brw


In [2]:
# ----------------------------------------------------------------------------------------- #
# B. Extracting the variables contained in the brw file
# ----------------------------------------------------------------------------------------- #
Variables = GetVarsHDF5( FILEBRW );
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
# Generating the new folders and files
# ----------------------------------------------------------------------------------------- #
PATHMAIN = joinpath( dirname( dirname( FILEBRW ) ), split( basename( FILEBRW ), "." )[ 1 ] );
PATHINFO = joinpath( PATHMAIN, "Info" );
PATHSTEP00 = joinpath( PATHMAIN, "STEP00" ); mkpath( PATHSTEP00 );
PATHFIGURES = joinpath( PATHMAIN, "Figures" );
PATHFIGURES_STEP00 = joinpath( PATHFIGURES, "STEP00" ); mkpath( PATHFIGURES_STEP00 );
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
FILESTEP00 = joinpath( PATHINFO, "STEP00.jld2" );
FILEPARAMETERS = joinpath( PATHINFO, "Parameters.jld2" );
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
# Constants
# ----------------------------------------------------------------------------------------- #
#= 
    All of the following values fall into the * category, 
    meaning that they can or should be modified
=#
# ----------------------------------------------------------------------------------------- #
limSat = 0.2; # Saturation limit of each channel before it is considered for discarding.
THR_EMP = 4000; # Threshold to be considered for spontaneous saturation ( μV )
# The maximum voltage value recorded by the device is 4125 μV.
Δt = 50; # ms for "Voltage Shift Deviation".
cm_ = :vik; # color map for all the plots
# font type for all the plots
plotfonts = Plots.font( pointsize = 10, family = "sans-serif" ); 
# ----------------------------------------------------------------------------------------- #

# -------------------------------------------------- Report -------------------------------------------------- #
File: ~/path-example/Data/BRWs/lesion.brw
Description: BRW-File Level3 - 3Brain RaW data file for high resolution MEA platform, HDF5-format - Created with BrainWave v.4.3.7542.24554 on date Thursday, September 17, 2020
HDF5 file size: 16.08 GB, corresponding to 300.0 seconds
Date of Analysis: Wed, 04 Sep 2024 15:15:00 by LBitn
You are now working on the new main path: /home/LBitn/path-example/Data/lesion
# ------------------------------------------------------------------------------------------------------------ #


In [4]:
# ----------------------------------------------------------------------------------------- #
# C. Setting the final size of each segment, in time and weight.
# ----------------------------------------------------------------------------------------- #
#= 
    MaxGB = Approximate size in GB for each segment, determined by the user. 
    4 sec by default. This value can be modified until an appropriate number of segments 
    is obtained. 
=#
MaxGB = 0.5;
N = GetChunkSize( Variables, MaxGB );
# ----------------------------------------------------------------------------------------- #

46 segments of 0.349 GB and 6.522 seconds each


In [5]:
# ----------------------------------------------------------------------------------------- #
# Pre-allocating the arrays to store the results
# ----------------------------------------------------------------------------------------- #
n0s = length( string( N ) );
Cardinality = Array{ Any }( undef, N ); fill!( Cardinality, [ ] );
VoltageShiftDeviation = Array{ Any }( undef, N ); fill!( VoltageShiftDeviation, [ ] );
Empties = Array{ Any }( undef, N ); fill!( Empties, [ ] );
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
for n = 1:N
    # ------------------------------------------------------------------------------------- #

    # ------------------------------------------------------------------------------------- #
    # D. Cutting one n-segment from N total segments
    BINRAW = OneSegment( Variables, n, N ); 
    nChs, nFrs = size( BINRAW );
    BINRAW = Digital2Analogue( Variables, BINRAW ); # Converting to μV
    # Defining new name for n-segment
    BINNAME = joinpath( PATHSTEP00, string( "BIN", lpad( n, n0s, "0" ), ".jld2" ) ); 
    # G. Saving n-segment with Float16 format
    jldsave( BINNAME; Data = Float16.( BINRAW ) );
    # ------------------------------------------------------------------------------------- #

    # ------------------------------------------------------------------------------------- #
    # Saturated channel detection -> Empty channels determination
    # ------------------------------------------------------------------------------------- #
    SatChs, SatFrs = SupThr( BINRAW, THR_EMP );
    PerSat = zeros( nChs );
    PerSat[ SatChs ] .= round.( length.( SatFrs ) ./ nFrs, digits = 2 );
    # E. First evaluation of emptie channels
    empties = [ ];
    empties = findall( PerSat .>= limSat ); 
    # ------------------------------------------------------------------------------------- #

    # ------------------------------------------------------------------------------------- #
    # Firts Evaluation, Cardinality and Voltage Shift Deviation
    # ------------------------------------------------------------------------------------- #
    CAR = [ ];
    VSD = [ ];
    CAR = UniqueCount( BINRAW );
    VSD = STDΔV( Variables, BINRAW, Δt );
    FirstEvaluation = Dict(
        "Cardinality"            => CAR,
        "VoltageShiftDeviation"  => VSD,
        "Empties"                => empties,
    );
    # ------------------------------------------------------------------------------------- #

    # ------------------------------------------------------------------------------------- #
    # F. First graph, Evaluation of the initial segment.
    # ------------------------------------------------------------------------------------- #
    aux = PatchEmpties( CAR, empties );
    P0 = plot( );
    P0 = Zplot( zscore( aux ), cm_, "\n" ^ 2 * "Cardinality of the Voltage" );
    aux = PatchEmpties( VSD, empties );
    P1 = plot( );
    P1 = Zplot( zscore( aux ), cm_, "\n" ^ 2 * "Voltage Shift Deviation" );
    P = plot( );
    P = plot(
        P0, P1,
        layout = ( 1, 2 ),
        wsize = ( 800, 400 )
        );
    title = plot( );
    title = plot(
        title = "\n" ^ 2 * "First evaluation, Raw Data",
        grid = false,
        showaxis = false,
        bottom_margin = -50Plots.px
        );
    F = plot( );
    F = plot( 
        title, P,
        layout = @layout( [ A{ 0.1h }; B{ 0.9h } ] ),
        wsize = ( 800, 500 ),
        titlefont = plotfonts,
    );
    FILEFIGURE = joinpath( PATHFIGURES_STEP00, string( "BIN", lpad( n, n0s, "0" ) ) );
    Plots.png( F, FILEFIGURE );
    # ------------------------------------------------------------------------------------- #

    # ------------------------------------------------------------------------------------- #
    Cardinality[ n ] = FirstEvaluation[ "Cardinality" ];
    VoltageShiftDeviation[ n ] = FirstEvaluation[ "VoltageShiftDeviation" ];
    Empties[ n ] = FirstEvaluation[ "Empties" ];
    # ------------------------------------------------------------------------------------- #
end
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
# G. Saving the results for all segments
# ----------------------------------------------------------------------------------------- #
Empties = sort( unique( vcat( Empties... ) ) );
step00 = Dict( 
    "Cardinality"           => Cardinality,
    "VoltageShiftDeviation" => VoltageShiftDeviation,
    "Empties"               => Empties,
    );
jldsave( FILESTEP00; Data = step00 );
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
# Saving the parameters for step-00
# ----------------------------------------------------------------------------------------- #
Parameters = Dict(
    "MaxGB"   => MaxGB,
    "limSat"  => limSat,
    "THR_EMP" => THR_EMP,
    "Δt"      => Δt,
    "cm_"     => cm_,
    "N"       => N
);
jldsave( FILEPARAMETERS; Data = Parameters );
# ----------------------------------------------------------------------------------------- #
# end of the step-00
# ----------------------------------------------------------------------------------------- #